# wk11 알테어 상호작용 과제

- 알테어 상호작용 노트북에서 작성했던 (연도에 대한 동적 쿼리가 포함된) 영화 산점도를 수정하여 `rule` 마크를 추가해 보라. 
- 이 `rule` 마크를 통하여 연도 `interval`로 선택한 연도의 데이터를 대상으로 평균 IMDB(또는 로튼 토마토) 등급을 보여주라.

In [1]:
import pandas as pd
import altair as alt

movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
df=pd.read_json(movies)
df

,Title,US_Gross,Worldwide_Gross,US_DVD_Sales,Production_Budget,Release_Date,MPAA_Rating,Running_Time_min,Distributor,Source,Major_Genre,Creative_Type,Director,Rotten_Tomatoes_Rating,IMDB_Rating,IMDB_Votes
0,The Land Girls,146083.0,146083.0,NaN,8000000.0,Jun 12 1998,R,NaN,Gramercy,None,None,None,None,NaN,6.1,1071.0
1,"First Love, Last Rites",10876.0,10876.0,NaN,300000.0,Aug 07 1998,R,NaN,Strand,None,Drama,None,None,NaN,6.9,207.0
2,I Married a Strange Person,203134.0,203134.0,NaN,250000.0,Aug 28 1998,None,NaN,Lionsgate,None,Comedy,None,None,NaN,6.8,865.0
3,Let's Talk About Sex,373615.0,373615.0,NaN,300000.0,Sep 11 1998,None,NaN,Fine Line,None,Comedy,None,None,13.0,NaN,NaN
4,Slam,1009819.0,1087521.0,NaN,1000000.0,Oct 09 1998,R,NaN,Trimark,Original Screenplay,Drama,Contemporary Fiction,None,62.0,3.4,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196,Zack and Miri Make a Porno,31452765.0,36851125.0,21240321.0,24000000.0,Oct 31 2008,R,101.0,Weinstein Co.,Original Screenplay,Comedy,Contemporary Fiction,Kevin Smith,65.0,7.0,55687.0
3197,Zodiac,33080084.0,83080084.0,20983030.0,85000000.0,Mar 02 2007,R,157.0,Paramount Pictures,Based on Book/Short Story,Thriller/Suspense,Dramatization,David Fincher,89.0,NaN,NaN
3198,Zoom,11989328.0,12506188.0,6679409.0,35000000.0,Aug 11 2006,PG,NaN,Sony Pictures,Based on Comic/Graphic Novel,Adventure,Super Hero,Peter Hewitt,3.0,3.4,7424.0
3199,The Legend of Zorro,45575336.0,141475336.0,NaN,80000000.0,Oct 28 2005,PG,129.0,Sony Pictures,Remake,Adventure,Historical Fiction,Martin Campbell,26.0,5.7,21161.0


In [2]:
# 아래 코드에서 '???' 부분을 수정하여 그림과 같은 시각화 결과를 완성하라. 

# - 지금까지 배운 것을 종합한 연습
#   - 앞에서 작성했던 (연도에 대한 동적 쿼리가 포함된) 영화 산점도를 수정하여 `rule` 마크를 추가해 보라. 
#   - 이 `rule` 마크를 통하여 연도 `interval`로 선택한 연도의 데이터를 대상으로 평균 IMDB(또는 로튼 토마토) 등급을 보여주라.

# x-축 (연도) 값으로 선택을 제한
brush = alt.selection_interval(
    encodings=['x']
    
)

# 동적 쿼리 도수분포도
years = alt.Chart(movies).mark_bar().add_selection(
    brush,
).encode(
    alt.X('year(Release_Date):T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)
# 선택 결과에 근거하여 투명도를 조절하는, 산점도를 base로 정의
base = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

# 산점도 base에 평균선을 layer로 겹쳐서 정의한 ratings
ratings = alt.layer(
    base,
    alt.Chart().mark_rule(color='#ff0000').encode( #색 설정: 빨강(#ff0000)
        x='average(Rotten_Tomatoes_Rating):Q'#Rotten_Tomatoes_Rating의 평균값을 x축으로 지정
        
    ).transform_filter(brush),#칫솔질 기능
    alt.Chart().mark_rule(color='#ff0000').encode(
        y='average(IMDB_Rating):Q'#IMDB_Rating의 평균값을 y축으로 지정
   ).transform_filter(brush), #칫솔질 기능
    
    data=movies #공통 데이터 지정
)
    

# 수직 병합
alt.vconcat(years, ratings).properties(spacing=5)

alt.VConcatChart(...)

In [4]:
#base + base.transform_regression('Rotten_Tomatoes_Rating:Q','IMDB_Rating:Q',).mark_line()